In [167]:
!pip install mediapipe

In [168]:
!pip install keyboard

In [169]:
import mediapipe as mp
import cv2
import time
import keyboard
# from google.colab.patches import cv2_imshow

In [170]:
import math

In [171]:
fileName = 'runningManWalkingMan.mp4'

In [203]:
cap = cv2.VideoCapture(fileName)

In [173]:
classes = [
    "run",
    "walk"
]

In [174]:
def calculateAngle(landmark1, landmark2, landmark3):
    '''
    This function calculates angle between three different landmarks.
    Args:
        landmark1: The first landmark containing the x,y and z coordinates.
        landmark2: The second landmark containing the x,y and z coordinates.
        landmark3: The third landmark containing the x,y and z coordinates.
    Returns:
        angle: The calculated angle between the three landmarks.

    '''

    # Get the required landmarks coordinates.
    x1 = landmark1.x
    y1 = landmark1.y
    x2 = landmark2.x
    y2 = landmark2.y
    x3= landmark3.x
    y3 = landmark3.y

    # Calculate the angle between the three points
    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))
    
    # Check if the angle is less than zero.
    if angle < 0:

        # Add 360 to the found angle.
        angle += 360
    
    # Return the calculated angle.
    return angle

In [272]:
def classifyPose(landmarks, mp_pose):
    '''
    This function classifies Ashton Hauls Running routine poses depending upon the angles of various body joints.
    Args:
        landmarks: A list of detected landmarks of the person whose pose needs to be classified.
        mp_pose: An mp.solutions.pose.
    Returns:
        label: The classified pose label of the person in the output_image.

    '''
    
    # Initialize the label of the pose. It is not known at this stage.
    label = 'Unknown Pose'

    # Specify the color (Red) with which the label will be written on the image.
    color = (0, 0, 255)
    
    # Calculate the required angles.
    #----------------------------------------------------------------------------------------------------------------
    
    # Get the angle between the left shoulder, elbow and wrist points. 
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
    # Get the angle between the right shoulder, elbow and wrist points. 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])   
    
    # Get the angle between the left elbow, shoulder and hip points. 
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

    # Get the angle between the right hip, shoulder and elbow points. 
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])

    # Get the angle between the left hip, knee and ankle points. 
    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])

    # Get the angle between the right hip, knee and ankle points 
    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    if (left_elbow_angle > 20 and left_elbow_angle < 100  or right_elbow_angle > 20 and right_elbow_angle < 100) or (left_elbow_angle > 20+180 and left_elbow_angle < 100+180  or right_elbow_angle > 20+180 and right_elbow_angle < 100+180):

    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the run pose.
    #----------------------------------------------------------------------------------------------------------------
            label = 'run'

            # Check if both legs are straight
            if (left_knee_angle > 160 and left_knee_angle < 195 and right_knee_angle < 160 and right_knee_angle > 195) or (left_knee_angle < 160 and left_knee_angle > 195 and right_knee_angle < 160 and right_knee_angle > 195):

                label = 'walk'
            else: 
                label = 'run'

    #----------------------------------------------------------------------------------------------------------------
    
    # Check if it is the walk pose.
    #----------------------------------------------------------------------------------------------------------------
    
     # Check if one leg is straight
    if left_knee_angle > 165 and left_knee_angle < 195 or right_knee_angle > 165 and right_knee_angle < 195:

        # Check if the shoulder at the required angle.
        if (left_shoulder_angle > 0 and left_shoulder_angle < 100 or left_shoulder_angle > 0+180 and left_shoulder_angle < 100+180) and (right_elbow_angle > 165 and right_elbow_angle < 195 or right_elbow_angle > 165+180 and right_elbow_angle < 195+180):

            # Specify the label of the drink pose.
            label = 'drink'                
    #----------------------------------------------------------------------------------------------------------------
    
    # Check if the pose is classified successfully
    if label != 'Unknown Pose':
        
        # Update the color (to green) with which the label will be written on the image.
        color = (0, 255, 0)  
    return label

In [268]:
cap = cv2.VideoCapture(fileName)

In [270]:
pTime = 0
cTime = 0

mpDraw = mp.solutions.drawing_utils
mpPose = mp.solutions.pose

pose = mpPose.Pose()
results = 0
while True:
  success, img = cap.read()
  if not success:
    cap.release()
    cv2.destroyAllWindows()
    break


  imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  results = pose.process(imgRGB)
  label = "unknown Pose"
    
  if results.pose_landmarks:
    label = classifyPose(results.pose_landmarks.landmark, mpPose)
    mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)
    h, w, c = img.shape
    for id, lm in enumerate(results.pose_landmarks.landmark):
      cx, cy = int(lm.x * w), int(lm.y * h)
      if id == 0:
        cv2.circle(img, (cx, cy), 5, (255,0, 0), cv2.FILLED)
    
    
  cTime = time.time()
  fps = 1 / (cTime - pTime)
  pTime = cTime

  cv2.putText(img, str(int(fps)), (70, 70), cv2.FONT_ITALIC, 1, (255,0,0))
  cv2.putText(img, label, (70, 10), cv2.FONT_ITALIC, 1, (0,0,255))#вывести текстом, что происходит в кадре
  cv2.imshow("Image", img)

  cv2.waitKey(1)

  if keyboard.is_pressed('space'):
    cv2.destroyAllWindows()
    break

In [177]:
 for id, lm in enumerate(results.pose_landmarks.landmark):
      # cx, cy = int(lm.x * w), int(lm.y * h)
      if id == 0:
        print(cx, cy)

174 945


In [178]:
# mpPose.PoseLandmark.RIGHT_SHOULDER.value

In [ ]:
# results.pose_landmarks.landmark[mpPose.PoseLandmark.RIGHT_SHOULDER.value].x

In [207]:
  calculateAngle(results.pose_landmarks.landmark[mpPose.PoseLandmark.LEFT_SHOULDER.value], results.pose_landmarks.landmark[mpPose.PoseLandmark.LEFT_ELBOW.value],results.pose_landmarks.landmark[mpPose.PoseLandmark.LEFT_WRIST.value])

157.5877600642503

In [262]:
calculateAngle(results.pose_landmarks.landmark[mpPose.PoseLandmark.LEFT_ELBOW.value],
                                         results.pose_landmarks.landmark[mpPose.PoseLandmark.LEFT_SHOULDER.value],
                                         results.pose_landmarks.landmark[mpPose.PoseLandmark.LEFT_HIP.value])

229.46335508594683

In [260]:
calculateAngle(results.pose_landmarks.landmark[mpPose.PoseLandmark.LEFT_HIP.value],
                                         results.pose_landmarks.landmark[mpPose.PoseLandmark.LEFT_KNEE.value],
                                         results.pose_landmarks.landmark[mpPose.PoseLandmark.LEFT_ANKLE.value])

186.1114647893238

In [264]:
calculateAngle(results.pose_landmarks.landmark[mpPose.PoseLandmark.RIGHT_SHOULDER.value],
                                         results.pose_landmarks.landmark[mpPose.PoseLandmark.RIGHT_ELBOW.value],
                                         results.pose_landmarks.landmark[mpPose.PoseLandmark.RIGHT_WRIST.value])

189.88914090398325